In [1]:
import os
from time import time
import numpy as np
import sys
sys.path.append('../')

from utils.codification_cnn import FitnessCNNParallel
from utils.codification_cutout import FitnessCutout, ChromosomeCutout
from GA.geneticAlgorithm import TwoLevelGA


Using TensorFlow backend.


In [2]:
data_folder = '../../datasets/MNIST_variations'
command = 'python3 ../train_gen.py'
verbose = 0

gpus = 2

In [3]:
fitness_cnn = FitnessCutout('../../exp_cifar10_time', use_resnet=True)    

(40000, 32, 32, 3) train samples
(10000, 32, 32, 3) validation samples
(10000, 32, 32, 3) test samples


In [4]:
c = ChromosomeCutout.random_individual()
print(c)


P:0.13|SL:0.59|SH:0.71|R1:0.16|R2:0.31|PL:False


In [5]:
# dataset params:
data_folder = data_folder
classes = []

# genetic algorithm params:
generations = -1
population_first_level = 10
population_second_level = 4
training_hours = 12
save_progress = True
maximize_fitness = False
statistical_validation = False
frequency_second_level = 3
start_level2 = 1

experiments_folder = '../../exp_cutout'
dataset='cifar10'

In [7]:
description = "\n----- Evolving cutout parameters -----\n"
experiments_folder = experiments_folder
os.makedirs(experiments_folder, exist_ok=True)

print("\nEVOLVING IN DATASET %s ...\n" % dataset)
exp_folder = os.path.join(experiments_folder, dataset)
folder = os.path.join(exp_folder, 'genetic')
fitness_folder = exp_folder
fitness_file = os.path.join(fitness_folder, 'fitness_example')   
os.makedirs(folder, exist_ok=True)

try:
    generational = TwoLevelGA.load_genetic_algorithm(folder=folder)

except:
    fitness_cnn.save(fitness_file)
    fitness = FitnessCNNParallel()
    fitness.set_params(chrom_files_folder=fitness_folder, fitness_file=fitness_file, max_gpus=gpus,
                   fp=32, main_line=command)
    generational = TwoLevelGA(chromosome=c,
                              fitness=fitness,
                              generations=generations,
                              population_first_level=population_first_level,
                              population_second_level=population_second_level,
                              training_hours=training_hours,
                              save_progress=save_progress,
                              maximize_fitness=maximize_fitness,
                              statistical_validation=statistical_validation,
                              folder=folder,
                              start_level2=start_level2,
                              frequency_second_level=frequency_second_level)
    generational.print_genetic(description)


ti_all = time()
print(generational.generation)
print(generational.num_generations)
if generational.generation < generational.num_generations:
    winner, best_fit, ranking = generational.evolve()
print("Total elapsed time: %0.3f" % (time() - ti_all))
    



EVOLVING IN DATASET cifar10 ...

Loading file ../../exp_cutout/cifar10/genetic/0_2020-01-31-13:19/GA_experiment
Number of individuals eliminated by age: 0
Genetic algorithm params
Number of generations: 10000
Population size: 10
Folder to save: ../../exp_cutout/cifar10/genetic/1_2020-01-31-13:56/GA_experiment
num parents: 2
offspring size: 8

Population size level one: 10
Population size level two: 4
Number of parents level one: 2
Number of parents level two: 2
Offspring size level one: 8
Offspring size level two: 2

----- Evolving cutout parameters -----

0
10000
Creating Initial population

Start evolution process...

Initial population initialization...10

0) Ranking level 1... Models to train: 10 ...OK (in 31.34 minutes)
Statistical validation in 0.00 minutes
Saving... Elapsed saved time: 0.002

1) Ranking level 1... Models to train: 8 ...

KeyboardInterrupt: 

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"


exp_folder = os.path.join(experiments_folder, dataset)
folder = os.path.join(exp_folder, 'genetic')


generational = TwoLevelGA.load_genetic_algorithm(folder=folder)
generational.finishing_evolution(show=True)


In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

winner = generational.best_individual['winner']
fit = generational.best_individual['best_fit']
test = generational.best_individual["test"]
print(winner)
print(1 - fit)
print(1 - test, test)

